In [1]:
pip install pandas_market_calendars

     |████████████████████████████████| 58 kB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 108 kB 3.8 MB/s eta 0:00:01
  Created wheel for trading-calendars: filename=trading_calendars-2.1.1-py3-none-any.whl size=140918 sha256=8d728fcf0107d1630e25a2bcad395a68a1767899f086c33d072bcb0abc0b4228
  Stored in directory: /root/.cache/pip/wheels/0d/3d/ae/394cfeb7b26c44a6c0bfdd42ab2f8ab899cefc00c4baa820f3
Successfully built trading-calendars
Note: you may need to restart the kernel to use updated packages.


In [2]:
import QuantConnect
from os import listdir, walk
from os.path import isfile, join
import pandas as pd
import pandas_market_calendars as mcal
import talib

In [3]:
path = '../../../../Data/forex/oanda/minute/'
currencies = ['audcad/'] #, 'audchf/']

In [4]:
for currency in currencies:
    files = [f for f in listdir(path + currency) if 2020 <= int(f.split('_')[0][:4]) <= 2022]

In [5]:
files[:5]

['20200102_quote.zip',
 '20200103_quote.zip',
 '20200105_quote.zip',
 '20200106_quote.zip',
 '20200107_quote.zip']

In [6]:
import datetime
def convert_to_time(t):
    t /= 1000
    seconds = int(t%60)
    t /= 60
    minutes = int(t%60)
    t /= 60
    hours = int(t%24)
    return datetime.time(hours, minutes, seconds)

In [7]:
# symbols = ["AUD", "CAD", "CHF", "EUR", "GBP", "NZD", "USD", "JPY"]
markets = {mcal.get_calendar('EUREX'): ['EUR', 'CHF', 'GBP'], mcal.get_calendar('HKEX'): ['JPY', 'AUD', 'NZD'], mcal.get_calendar('XMEX'): ['CAD', 'USD']}
def market_open(symbol, sch, d):
    symbol = symbol[:3].upper()
    market = None
    for k, v in markets.items():
         if symbol in v:
             market = k
    return cal.open_at_time(sch, d)

In [8]:
import talib

CANDLESTICKS = talib.__function_groups__['Pattern Recognition']
INDICATORS = {
    'BBANDS': {'method': 'BBANDS', 'period': 20}, 'SMA_SLOW': {'method': 'SMA', 'period': 200},
    'SMA_FAST': {'method': 'SMA', 'period': 50}, 'EMA_SLOW': {'method': 'EMA', 'period': 20},
    'EMA_FAST': {'method': 'EMA', 'period': 9}, 'ATR': {'method': 'ATR', 'period': 14},
    'RSI': {'method': 'RSI', 'period': 14}
}


def get_indicators(df):
    df = df.copy()
    opens = df['open'].values
    highs = df['high'].values
    lows = df['low'].values
    closes = df['close'].values
    # Add indicator columns to DataFrame
    for indicator_name, indicator_values in INDICATORS.items():
        if indicator_name == 'BBANDS':
            ub, _, lb = getattr(talib, indicator_values['method'])(closes, indicator_values['period'])
            df[f'{indicator_name}_UB'], df[f'{indicator_name}_LB'] = ub, lb
        elif indicator_name == 'ATR':
            df[indicator_name] = getattr(talib, indicator_values['method'])(highs,lows,closes,indicator_values['period'])
        else:
            df[indicator_name] = getattr(talib, indicator_values['method'])(closes, indicator_values['period'])
    return df


def get_candlestick_markers(df):
    df = df.copy()
    for method in CANDLESTICKS:
        df[method] = getattr(talib, method)(df["open"].values, df["high"].values, df["low"].values,
                                              df["close"].values)
    return df

In [9]:
k, d = talib.STOCHRSI

TypeError: 'function' object is not iterable

In [10]:
def get_fractal(high, low, n=2):
    if (high[n-2] < high[n]) and (high[n-1] < high[n]) and (high[n+1] < high[n]) and (high[n+2] < high[n]):
        return -1
    elif (low[n-2] > low[n]) and (low[n-1] > low[n]) and (low[n+1] > low[n]) and (low[n+2] > low[n]):
        return 1
    else:
        return 0

In [44]:
import numpy as np
def fractals(df, column_name_high='fractals_high', column_name_low='fractals_low'):
    """
    Fractals
    ---------
        https://www.metatrader4.com/en/trading-platform/help/analytics/tech_indicators/fractals
        >>> Indicators.fractals(column_name_high='fractals_high', column_name_low='fractals_low')
        :param str column_name_high: Column name for High values, default: fractals_high
        :param str column_name_low: Column name for Low values, default: fractals_low
        :return: None
    """
    df_tmp = df[['high', 'low']]
    df_tmp = df_tmp.assign(fh=np.where(
        (df_tmp['high'] > df_tmp['high'].shift(1)) &
        (df_tmp['high'] > df_tmp['high'].shift(2)) &
        (df_tmp['high'] > df_tmp['high'].shift(-1)) &
        (df_tmp['high'] > df_tmp['high'].shift(-2)),
        True, False
    ))
    df_tmp = df_tmp.assign(fl=np.where(
        (df_tmp['low'] < df_tmp['low'].shift(1)) &
        (df_tmp['low'] < df_tmp['low'].shift(2)) &
        (df_tmp['low'] < df_tmp['low'].shift(-1)) &
        (df_tmp['low'] < df_tmp['low'].shift(-2)),
        True, False
    ))
    df_tmp = df_tmp[['fh', 'fl']]
    df_tmp = df_tmp.rename(columns={'fh': column_name_high, 'fl': column_name_low})
    return df.merge(df_tmp, left_index=True, right_index=True)

In [45]:
t = temp.copy()
t['high'] = t['high_bid']
t['low'] = t['low_bid']

In [47]:
t = fractals(t)

In [50]:
t.tail()

,time,open_bid,high_bid,low_bid,close_bid,open_ask,high_ask,low_ask,close_ask,ema1,...,k,d,fractal,date,datetime,spread,high,low,fractals_high,fractals_low
475,07:55:00,0.96579,0.96583,0.96570,0.96571,0.96598,0.96605,0.96588,0.96591,0.965753,...,0.000000,66.666667,0.0,20210322,2021-03-22 07:55:00+00:00,0.00020,0.96583,0.96570,False,False
476,07:56:00,0.96571,0.96587,0.96567,0.96585,0.96591,0.96609,0.96588,0.96603,0.965779,...,100.000000,66.666667,0.0,20210322,2021-03-22 07:56:00+00:00,0.00018,0.96587,0.96567,False,False
477,07:57:00,0.96585,0.96590,0.96561,0.96561,0.96603,0.96610,0.96581,0.96582,0.965783,...,0.000000,33.333333,0.0,20210322,2021-03-22 07:57:00+00:00,0.00021,0.96590,0.96561,True,False
478,07:58:00,0.96561,0.96574,0.96555,0.96569,0.96582,0.96591,0.96575,0.96587,0.965792,...,19.772225,39.924075,0.0,20210322,2021-03-22 07:58:00+00:00,0.00018,0.96574,0.96555,False,False
479,07:59:00,0.96569,0.96579,0.96565,0.96574,0.96587,0.96597,0.96581,0.96593,0.965805,...,100.000000,39.924075,0.0,20210322,2021-03-22 07:59:00+00:00,0.00019,0.96579,0.96565,False,False


In [52]:
temp.tail()

,time,open_bid,high_bid,low_bid,close_bid,open_ask,high_ask,low_ask,close_ask,ema1,ema2,ema3,k,d,fractal,date,datetime,spread
475,07:55:00,0.96579,0.96583,0.96570,0.96571,0.96598,0.96605,0.96588,0.96591,0.965753,0.965718,0.965753,0.000000,66.666667,0.0,20210322,2021-03-22 07:55:00+00:00,0.00020
476,07:56:00,0.96571,0.96587,0.96567,0.96585,0.96591,0.96609,0.96588,0.96603,0.965779,0.965730,0.965758,100.000000,66.666667,0.0,20210322,2021-03-22 07:56:00+00:00,0.00018
477,07:57:00,0.96585,0.96590,0.96561,0.96561,0.96603,0.96610,0.96581,0.96582,0.965783,0.965733,0.965759,0.000000,33.333333,0.0,20210322,2021-03-22 07:57:00+00:00,0.00021
478,07:58:00,0.96561,0.96574,0.96555,0.96569,0.96582,0.96591,0.96575,0.96587,0.965792,0.965739,0.965762,19.772225,39.924075,0.0,20210322,2021-03-22 07:58:00+00:00,0.00018
479,07:59:00,0.96569,0.96579,0.96565,0.96574,0.96587,0.96597,0.96581,0.96593,0.965805,0.965746,0.965765,100.000000,39.924075,0.0,20210322,2021-03-22 07:59:00+00:00,0.00019


In [24]:
train = pd.DataFrame()
train_close = pd.DataFrame()
test = pd.DataFrame()
sizes = []
symbol = currencies[0]
for i, f in enumerate(files):
    if (i+1) % 100 == 0:
        print(i)
    date = f.split('_')[0]
    temp = pd.read_csv(path+currency+f, names=['time', 'open_bid', 'high_bid', 'low_bid', 'close_bid', 'volume', 'open_ask', 'high_ask', 'low_ask', 'close_ask', 'volume2'])
    sizes.append(temp.shape[0])
    if temp.shape[0] < 1400:
        continue
    temp['ema1'] = talib.EMA(temp['close_ask'], 20)
    temp['ema2'] = talib.EMA(temp['close_ask'], 50)
    temp['ema3'] = talib.EMA(temp['close_ask'], 100)
    temp['k'], temp['d'] = talib.STOCHRSI(temp['close_ask'], 14, 3, 3, 0)
    fs = [None, None, None, None, None, None, None, None]
    for i in range(8, len(temp)):
        fs.append(get_fractal(temp['high_ask'].iloc[:i-3].values, temp['low_ask'].iloc[:i-3].values))
    temp['fractal'] = fs
    temp.dropna(inplace=True)
    del temp['volume']
    del temp['volume2']
    temp['date'] = date
    temp['time'] = temp['time'].apply(convert_to_time)
    temp['datetime'] = temp['time'].apply(lambda x: datetime.datetime(int(date[:4]), int(date[4:6]), int(date[6:]), x.hour, x.minute, x.second, tzinfo=datetime.timezone.utc))
    
    symbol = symbol[:3].upper()
    market = None
    for k, v in markets.items():
         if symbol in v:
             market = k
    if market is None:
        continue
    sch = market.schedule(temp['datetime'].iloc[0].date(), temp['datetime'].iloc[0].date())
    temp['market_open'] = temp['datetime'].apply(lambda d: market.open_at_time(sch, d))
#     print(temp[temp['market_open'] == False]['close_bid'].pct_change(90).dropna().mean(), temp[temp['market_open'] == True]['close_bid'].pct_change(90).dropna().mean())
    train_close = pd.concat([train_close, temp[temp['market_open'] == False]])
    temp = temp[temp['market_open'] == True]
    del temp['market_open']
    temp['spread'] = temp['close_ask'] - temp['close_bid']
    
    if f.startswith('2020'):
        train = pd.concat([train, temp])
    else:
        test = pd.concat([test, temp])
#     break

99
199
299


In [53]:
datetime.time?

Init signature: datetime.time(self, /, *args, **kwargs)
Docstring:     
time([hour[, minute[, second[, microsecond[, tzinfo]]]]]) --> a time object

All arguments are optional. tzinfo may be None, or an instance of
a tzinfo subclass. The remaining arguments may be ints.
File:           /opt/miniconda3/lib/python3.6/datetime.py
Type:           type
Subclasses:     


In [25]:
train['close_bid'].pct_change(90).dropna().mean(), train_close['close_bid'].pct_change(90).dropna().mean()

(0.00011250177790364594, 2.4042854844508305e-05)

In [61]:
train.set_index('datetime', inplace=True)
test.set_index('datetime', inplace=True)

In [62]:
len(train), len(test)

(63231, 14124)

In [63]:
train

,time,open_bid,high_bid,low_bid,close_bid,open_ask,high_ask,low_ask,close_ask,ema1,ema2,ema3,k,d,fractal,date,spread
datetime,,,,,,,,,,,,,,,,,
2020-01-02 01:39:00+00:00,01:39:00,0.91021,0.91023,0.91011,0.91012,0.91050,0.91051,0.91040,0.91040,0.910618,0.910611,0.910605,0.000000e+00,2.368476e-14,0.0,20200102,0.00028
2020-01-02 01:40:00+00:00,01:40:00,0.91012,0.91013,0.91005,0.91005,0.91041,0.91041,0.91033,0.91033,0.910590,0.910600,0.910600,0.000000e+00,2.368476e-14,0.0,20200102,0.00028
2020-01-02 01:41:00+00:00,01:41:00,0.91007,0.91008,0.90997,0.90999,0.91033,0.91034,0.91021,0.91026,0.910559,0.910586,0.910593,0.000000e+00,2.368476e-14,0.0,20200102,0.00027
2020-01-02 01:42:00+00:00,01:42:00,0.90999,0.91005,0.90999,0.91005,0.91027,0.91030,0.91025,0.91028,0.910532,0.910574,0.910587,5.901723e+01,1.967241e+01,0.0,20200102,0.00023
2020-01-02 01:43:00+00:00,01:43:00,0.91004,0.91006,0.91000,0.91006,0.91029,0.91035,0.91028,0.91035,0.910515,0.910566,0.910582,1.000000e+02,5.300574e+01,0.0,20200102,0.00029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-30 07:55:00+00:00,07:55:00,0.97892,0.97904,0.97891,0.97895,0.97922,0.97930,0.97916,0.97922,0.979191,0.979244,0.979283,3.124987e-13,6.504644e+00,0.0,20201230,0.00027
2020-12-30 07:56:00+00:00,07:56:00,0.97895,0.97904,0.97890,0.97898,0.97922,0.97933,0.97921,0.97929,0.979200,0.979246,0.979283,1.000000e+02,3.333333e+01,0.0,20201230,0.00031
2020-12-30 07:57:00+00:00,07:57:00,0.97898,0.97901,0.97887,0.97895,0.97929,0.97930,0.97919,0.97926,0.979206,0.979246,0.979282,5.057983e+01,5.019328e+01,0.0,20201230,0.00031


In [64]:
train.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 63231 entries, 2020-01-02 01:39:00+00:00 to 2020-12-30 07:59:00+00:00
Data columns (total 17 columns):
time         63231 non-null object
open_bid     63231 non-null float64
high_bid     63231 non-null float64
low_bid      63231 non-null float64
close_bid    63231 non-null float64
open_ask     63231 non-null float64
high_ask     63231 non-null float64
low_ask      63231 non-null float64
close_ask    63231 non-null float64
ema1         63231 non-null float64
ema2         63231 non-null float64
ema3         63231 non-null float64
k            63231 non-null float64
d            63231 non-null float64
fractal      63231 non-null float64
date         63231 non-null object
spread       63231 non-null float64
dtypes: float64(15), object(2)
memory usage: 8.7+ MB


In [209]:
train.to_csv(f"data/train/5_15_30_1h_timeframes.csv")
test.to_csv(f"data/test/5_15_30_1h_timeframes.csv")

In [127]:
class Strategy:
    long = 1
    short = -1
    margin = 40
    
    position_size = 0
    entry_name = None
    entry_close = None
    exit_name = None
    close_bid = None
    close_ask = None
    limit = None
    stop = None
    
    def entry(self, name, direction, amount):
        self.position_size = amount * direction
        self.entry_name = name
        if direction == 1:
            self.entry_close = self.close_ask
        elif direction == -1:    
            self.entry_close = self.close_bid
    def exit(self, name, entry_name, limit=None, stop=None):
        if self.entry_name == None or entry_name != self.entry_name:
            return None
        if limit is not None:
            self.limit = limit
        if stop is not None:
            self.stop = stop
        self.exit_name = name
    def step(self, close_bid, close_ask):
        self.close_bid = close_bid
        self.close_ask = close_ask
        result = None
        if self.position_size == 0:
            return None
        elif self.position_size == 1:
            if self.limit is not None and close_bid >= self.limit:
                result = self.limit - self.entry_close
                result /= self.entry_close
                result = self.margin * result
                self.reset()
            elif self.stop is not None and close_bid < self.stop:
                result = self.entry_close - self.stop
                result /= self.entry_close
                result = self.margin * result
                self.reset()
        elif self.position_size == -1:
            if self.limit is not None and close_ask >= self.limit:
                result = self.entry_close - self.limit
                result /= self.entry_close
                result = self.margin * result
                self.reset()
            elif self.stop is not None and close_ask < self.stop:
                result = self.stop - self.entry_close
                result /= self.entry_close
                result = self.margin * result
                self.reset()
        else:
            return None
        
        return result
                
                
    def reset(self):
        self.position_size = 0
        self.entry_name = None
        self.entry_close = None
        self.exit_name = None
        self.limit = None
        self.stop = None
        

In [128]:
strategy = Strategy()
RISK = 1.5
profits = []
for date, row in train.iterrows():
    profits.append(strategy.step(row['close_bid'], row['close_ask']))
    if profits[-1] is not None:
        print(profits[-1])
    ProfitTargets = []
    StopTargets = []
    if strategy.position_size == 0:
        if row['d'] < 20 and row['k'] < 20 and row['fractal'] == True and row['ema1'] > row['ema2'] and row['ema2'] > row['ema3'] and row['close_ask'] > row['ema3']:
            if row['close_ask'] < row['ema2']:
                ProfitTarget = (row['close_ask'] - row['ema3']) * RISK
                if ProfitTarget > row['spread'] * RISK:
                    StopTarget = row['ema3']
                    strategy.entry("buy3", strategy.long, 1.) 
                    strategy.exit("profit3", "buy3", limit = row['close_ask'] + ProfitTarget)
                    strategy.exit("stop3", "buy3", stop = StopTarget)
            elif row['close_ask'] < row['ema1']:
                ProfitTarget = (row['close_ask'] - row['ema2']) * RISK
                if ProfitTarget > row['spread'] * RISK:
                    StopTarget = row['ema2']
                    strategy.entry("buy2", strategy.long, 1.) 
                    strategy.exit("profit2", "buy2", limit = row['close_ask'] + ProfitTarget)
                    strategy.exit("stop2", "buy2", stop = StopTarget)
        if row['d'] > 80 and row['k'] > 80 and row['fractal'] == True and row['ema1'] < row['ema2'] and row['ema2'] < row['ema3'] and row['close_bid'] < row['ema3']:
            if row['close_bid'] > row['ema2']:
                ProfitTarget = (row['ema3'] - row['close_bid']) * RISK
                if ProfitTarget > row['spread'] * RISK:
                    StopTarget =row['ema3']
                    strategy.entry("sell3", strategy.short, 1.) 
                    strategy.exit("profit3", "sell3", limit = row['close_bid'] - ProfitTarget)
                    strategy.exit("stop3", "sell3", stop = StopTarget)
            elif row['close_bid'] > row['ema1']:
                ProfitTarget = (row['ema2'] - row['close_bid']) * RISK
                if ProfitTarget > row['spread'] * RISK:
                    StopTarget = row['ema2']
                    strategy.entry("sell2", strategy.short, 1.) 
                    strategy.exit("profit2", "sell2", limit = row['close_bid'] - ProfitTarget)
                    strategy.exit("stop2", "sell2", stop = StopTarget)

0.06668582395247372
0.0618422260941717
0.14933580799941
0.1408651882527399
0.1267199570709202
0.07498602450481312
0.04679475580965812
0.06635611359473903
0.02875293913263972
0.021139602838026236
0.01954071744055557
0.018211418361486747
0.020556158419943904
0.00980934909166958
0.013191722506370445
0.009982727565690847
0.04291116363248677
0.031090919721637696
0.01479151509600157
0.02258136451853268
0.0149777530246511
0.05670649338848141
0.02358503855821363
0.023368185769985735
0.017591210874881597
0.0265355046955884
0.053031191291957214
0.05172173574786808
0.0484071920851826
0.042738775788287206
0.04286554385368672
0.03819031825400325
0.024797070448295072
0.02209681005841636
0.011433349627068651


In [117]:
sum([p for p in profits if p is not None])

-0.0037000000000000366

In [120]:
sum([p for p in profits if p is not None])

0.027192546058762646